#### **Fundamentos de Bancos de Dados Relacionais e NoSQL**

#### **Conteúdo - Bases e Notebook da aula**

https://github.com/FIAP/Pos_Tech_DTAT/tree/main/Fase%203

#### **Importação de pacotes, bibliotecas e funções (def)**

In [1]:
# Importar biblioteca completa
import boto3
import pandas as pd
import os
import plotly.express as px
import requests
import botocore
import psycopg2
import numpy as np
import duckdb

# Importar função especifica de um módulo
from botocore.exceptions import BotoCoreError, ClientError
from sqlalchemy import create_engine, text, inspect
from dotenv import load_dotenv
from io import StringIO
from datetime import datetime


In [2]:
# Testar a conexão ao banco de dados
def test_connection(engine):

    try:
        with engine.connect() as connection:
            
            # Testar a versão do PostgreSQL
            result = connection.execute(text("SELECT version();"))
            versao = result.fetchone()
            print("✅ Conectado com sucesso:", versao[0])

            # Listar as tabelas no schema público
            result = connection.execute(text("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public';
            """))
            tabelas = result.fetchall()
            print("📄 Tabelas no banco:")
            for tabela in tabelas:
                print("  -", tabela[0])

    except Exception as e:
        print("❌ Erro ao executar comandos:", e)


#### **Instalação do AWS CLI (Opcional, mas Recomendado)**

Para rodar comandos da AWS no terminal para testar credenciais, criar buckets ou consultar recursos instale o **AWS CLI v2**.

##### **macOS / Linux (zsh ou bash)**

**Instalação com Homebrew (recomendado para macOS)**
```bash
brew install awscli
```

##### **Instalação manual**
```bash
curl "https://awscli.amazonaws.com/AWSCLIV2.pkg" -o "AWSCLIV2.pkg"
sudo installer -pkg AWSCLIV2.pkg -target /
```

##### **Verificar instalação**
```bash
aws --version
```

##### **Windows**

1. Baixe o instalador: [AWS CLI v2](https://awscli.amazonaws.com/AWSCLIV2.msi)
2. Execute o instalador e finalize.
3. Teste no PowerShell:
```bash
aws --version
```

##### **Testar credenciais**
Após instalar o **AWS CLI**, teste o perfil `academy` criado:

```bash
aws sts get-caller-identity --profile academy
```

Se retornar o **Account ID** e o **ARN**, as credenciais estão funcionando ✅

#### **Credenciais do AWS Academy** 

Para seguir com as etapas desse notebook será necessario pegar as credencias do AWS Academy e criar o arquivo com essas credencias:

**1. Pegar credenciais do AWS Academy**  
1. Acesse [AWS Academy](https://awsacademy.instructure.com/)
2. Acesse a opção **Cursos** no lado esquerdo e acesse o seu respectivo curso
3. Acessa a opçãp **Módulos**
4. Acesse a opção **Iniciar os laboratórios de aprendizagem da AWS Academy**
5. Clique em **Start Lab**
6. Clique em **AWS Details**
7. Clique em **AWS CLI: Show** e copie:
   - **AWS Access Key ID**
   - **AWS Secret Access Key**
   - **Session Token**

⚠️ As credenciais são temporárias (válidas por 3 horas).

**2. Criar arquivo de credenciais**  

Crie o arquivo **`credentials`** no caminho:

- **macOS/Linux:** `~/.aws/credentials`
- **Windows:** `%USERPROFILE%\.aws\credentials`

⚠️ Salvar o arquivo sem formato --> Na opçãp Tipo deixar Todos os arquivos

**3. Conteúdo do arquivo**
```ini
[academy]
aws_access_key_id = SUA_ACCESS_KEY
aws_secret_access_key = SUA_SECRET_KEY
aws_session_token = SEU_SESSION_TOKEN
region = sa-east-1
output = json
```

**4. Testar configuração**
```bash
aws sts get-caller-identity --profile academy
```

Se tudo estiver certo, rode o notebook.  
Agora ele **usa automaticamente o perfil `academy`** para se conectar à AWS.

#### **Testar conexão AWS via Python**

In [3]:
# Validar conexão
try:
    session = boto3.Session(profile_name="academy")
    sts = session.client("sts")
    identity = sts.get_caller_identity()
    print("✅ Conectado à conta\n")
    print("UserId:", identity["UserId"])
    print("Account:", identity["Account"])
    print("Arn:", identity["Arn"])

except (BotoCoreError, ClientError) as e:
    print("❌ Erro ao conectar à AWS. Verifique suas credenciais e tente novamente.")
    print("Detalhes do erro:", e)


✅ Conectado à conta

UserId: AROAWIUHO6CVULDIIR3FL:user4308167=ricardviana1@gmail.com
Account: 430854566059
Arn: arn:aws:sts::430854566059:assumed-role/voclabs/user4308167=ricardviana1@gmail.com


#### **Configuração do PostgreSQL na AWS RDS**

##### 1. **Criar instância RDS com PostgreSQL (SandBox)**

1. Acesse o console AWS → [https://us-east-1.console.aws.amazon.com/rds/home?region=us-east-1#](https://console.aws.amazon.com/rds/)
2. Clique em **Criar banco de dados**
3. Selecione:
   - **Tipo de banco:** PostgreSQL
   - **Versão:** PostgreSQL 15 (ou mais recente)
   - **Modelo de uso:** SandBox
   - **Identificador da instância:** `postgres-db`
   - **Usuário:** `postgres`
   - **Senha:** crie uma senha segura
4. Tipo de instância: `db.t3.micro`
5. Armazenamento: 20 GB (SSD General Purpose)
6. **Acesso público:** Habilitado (Sim)
7. **Nome do banco de dados inicial:** `db_relacional`
8. Clique em **Criar banco de dados**

##### **2. Liberar o IP na VPC / Grupo de Segurança (Security Group)**

1. Vá para **EC2 > Grupos de Segurança**
2. Encontre o grupo associado à instância RDS
3. Clique em **Editar regras de entrada**
4. Adicione uma nova regra:
   - Tipo: `PostgreSQL`
   - Porta: `5432`
   - Origem: `Seu IP` (ou `0.0.0.0/0` temporariamente para teste – cuidado com isso em produção)
5. Salve as alterações.

✅ Agora o acesso externo ao banco estará liberado para seu IP

##### **3. Copie o Endpoint da RDS**

1. Volte ao RDS > Banco de dados > `bd-relacional`
2. Copie o valor do campo **Endpoint** (algo como `bd-relacional.xxxxxx.us-east-1.rds.amazonaws.com`)
3. Use esse endpoint no notebook para se conectar com o PostgreSQL

#### **Aula 1 - Introdução ao Banco De Dados Relacionais**

##### **Conectar ao PostgreSQL via RDS + Executar Comandos SQL**

In [4]:
# Criar a engine para conexão ao banco de dados usando .env

load_dotenv()

usuario = os.getenv("POSTGRES_USER")
senha = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")
porta = os.getenv("POSTGRES_PORT")
banco = os.getenv("POSTGRES_DB")

engine = create_engine(f"postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}")

In [5]:
# Testar a conexão
test_connection(engine)

✅ Conectado com sucesso: PostgreSQL 17.4 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 12.4.0, 64-bit
📄 Tabelas no banco:
  - dim_cliente
  - dim_produto
  - fato_pedidos
  - itens_pedido
  - pedidos
  - produtos
  - clientes
  - tipos_produto


In [6]:
# Dropar as tabelas --> CASCADE garante que todas as dependências (FKs) sejam eliminadas junto com a tabela
drop_script = """
DROP TABLE IF EXISTS itens_pedido CASCADE;
DROP TABLE IF EXISTS pedidos CASCADE;
DROP TABLE IF EXISTS produtos CASCADE;
DROP TABLE IF EXISTS clientes CASCADE;
DROP TABLE IF EXISTS tipos_produto CASCADE;
"""

conn = engine.raw_connection()
try:
    cursor = conn.cursor()
    cursor.execute(drop_script)
    conn.commit()
    print("🗑️ Todas as tabelas foram deletadas com sucesso.")
finally:
    cursor.close()
    conn.close()

# Listar de comandos individuais
ddl_commands = [
    """
    CREATE TABLE IF NOT EXISTS tipos_produto (
      id_tipo SERIAL PRIMARY KEY,
      nome_tipo VARCHAR(50) NOT NULL
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS produtos (
      id_produto SERIAL PRIMARY KEY,
      nome_produto VARCHAR(100) NOT NULL,
      preco DECIMAL(10,2) NOT NULL,
      id_tipo INT REFERENCES tipos_produto(id_tipo)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS clientes (
      id_cliente SERIAL PRIMARY KEY,
      nome VARCHAR(100) NOT NULL,
      email VARCHAR(100),
      telefone VARCHAR(20), 
      cidade VARCHAR(100) NOT NULL, 
      estado VARCHAR(2) NOT NULL
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS pedidos (
      id_pedido SERIAL PRIMARY KEY,
      data_pedido DATE NOT NULL,
      status VARCHAR(20) NOT NULL,
      id_cliente INT NOT NULL REFERENCES clientes(id_cliente)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS itens_pedido (
    id_item SERIAL PRIMARY KEY,
    id_pedido INT NOT NULL,
    id_produto INT NOT NULL,
    quantidade INT NOT NULL,
    preco_unitario DECIMAL(10,2) NOT NULL,
    CONSTRAINT fk_pedido FOREIGN KEY (id_pedido) REFERENCES pedidos(id_pedido) ON DELETE CASCADE,
    CONSTRAINT fk_produto FOREIGN KEY (id_produto) REFERENCES produtos(id_produto) ON DELETE CASCADE
    );

    """
]

# Lista de nomes das tabelas na mesma ordem dos comandos
table_names = [
    "tipos_produto",
    "produtos",
    "clientes",
    "pedidos",
    "itens_pedido"
]

with engine.begin() as conn:
    for cmd, table in zip(ddl_commands, table_names):
        conn.execute(text(cmd))
        print(f"✅ Tabela '{table}' criada com sucesso!")

🗑️ Todas as tabelas foram deletadas com sucesso.
✅ Tabela 'tipos_produto' criada com sucesso!
✅ Tabela 'produtos' criada com sucesso!
✅ Tabela 'clientes' criada com sucesso!
✅ Tabela 'pedidos' criada com sucesso!
✅ Tabela 'itens_pedido' criada com sucesso!


In [7]:
# Verificar as chaves primarias
df_pks = pd.read_sql_query("""
SELECT 
    kcu.table_schema,
    kcu.table_name,
    kcu.column_name,
    tc.constraint_name
FROM information_schema.table_constraints tc
JOIN information_schema.key_column_usage kcu
  ON tc.constraint_name = kcu.constraint_name
WHERE tc.constraint_type = 'PRIMARY KEY'
  AND kcu.table_schema = 'public';
""", con=engine)

df_pks.head(10)

,table_schema,table_name,column_name,constraint_name
0,public,tipos_produto,id_tipo,tipos_produto_pkey
1,public,produtos,id_produto,produtos_pkey
2,public,clientes,id_cliente,clientes_pkey
3,public,pedidos,id_pedido,pedidos_pkey
4,public,dim_cliente,id_cliente,dim_cliente_pkey
5,public,dim_produto,id_produto,dim_produto_pkey
6,public,fato_pedidos,id_pedido,fato_pedidos_pkey
7,public,itens_pedido,id_item,itens_pedido_pkey


In [8]:
# Verificar chaves estrangeiras e relacionamento
df_fks = pd.read_sql_query("""
SELECT 
    tc.table_name AS tabela_origem,
    kcu.column_name AS coluna_origem,
    ccu.table_name AS tabela_referenciada,
    ccu.column_name AS coluna_referenciada
FROM information_schema.table_constraints AS tc
JOIN information_schema.key_column_usage AS kcu
  ON tc.constraint_name = kcu.constraint_name
JOIN information_schema.constraint_column_usage AS ccu
  ON ccu.constraint_name = tc.constraint_name
WHERE tc.constraint_type = 'FOREIGN KEY'
  AND tc.table_schema = 'public';
""", con=engine)

df_fks.head()


,tabela_origem,coluna_origem,tabela_referenciada,coluna_referenciada
0,produtos,id_tipo,tipos_produto,id_tipo
1,fato_pedidos,id_cliente,dim_cliente,id_cliente
2,fato_pedidos,id_produto,dim_produto,id_produto
3,pedidos,id_cliente,clientes,id_cliente
4,itens_pedido,id_pedido,pedidos,id_pedido


In [9]:
# Realizar join entre as tabelas
df_relacionamentos = pd.read_sql_query("""
SELECT 
  tc.constraint_name,
  tc.table_name AS origem,
  kcu.column_name AS coluna_origem,
  ccu.table_name AS destino,
  ccu.column_name AS coluna_destino
FROM information_schema.table_constraints AS tc
JOIN information_schema.key_column_usage AS kcu
  ON tc.constraint_name = kcu.constraint_name
JOIN information_schema.constraint_column_usage AS ccu
  ON ccu.constraint_name = tc.constraint_name
WHERE tc.constraint_type = 'FOREIGN KEY'
ORDER BY origem;
""", con=engine)

df_relacionamentos.head()


,constraint_name,origem,coluna_origem,destino,coluna_destino
0,fato_pedidos_id_cliente_fkey,fato_pedidos,id_cliente,dim_cliente,id_cliente
1,fato_pedidos_id_produto_fkey,fato_pedidos,id_produto,dim_produto,id_produto
2,fk_pedido,itens_pedido,id_pedido,pedidos,id_pedido
3,fk_produto,itens_pedido,id_produto,produtos,id_produto
4,pedidos_id_cliente_fkey,pedidos,id_cliente,clientes,id_cliente


In [10]:
# Executar arquivos .sql
truncate_script = """
TRUNCATE TABLE itens_pedido CASCADE;
TRUNCATE TABLE pedidos CASCADE;
TRUNCATE TABLE produtos CASCADE;
TRUNCATE TABLE clientes CASCADE;
TRUNCATE TABLE tipos_produto CASCADE;
"""

github_urls = [
    "https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/refs/heads/main/Fase%203/Aula4/sql/aula4/tipos_produto.sql",
    "https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/refs/heads/main/Fase%203/Aula4/sql/aula4/produtos.sql",
    "https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/refs/heads/main/Fase%203/Aula4/sql/aula4/clientes.sql",
    "https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/refs/heads/main/Fase%203/Aula4/sql/aula4/pedidos.sql",
    "https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/refs/heads/main/Fase%203/Aula4/sql/aula4/itens_pedido.sql"
]

with engine.begin() as conn:
    conn.execute(text(truncate_script))
    print("🗑️ Dados apagados de todas as tabelas.")

    for url in github_urls:
        response = requests.get(url)
        response.raise_for_status()
        sql_content = response.text
        conn.execute(text(sql_content))
        print(f"✅ Executado: {url.split('/')[-1]}")

🗑️ Dados apagados de todas as tabelas.
✅ Executado: tipos_produto.sql
✅ Executado: produtos.sql
✅ Executado: clientes.sql
✅ Executado: pedidos.sql
✅ Executado: itens_pedido.sql


In [11]:
# Join para analise de vendas
df = pd.read_sql_query(

"""
                       
SELECT 
  c.nome AS cliente,
  p.data_pedido as data_pedido,
  pr.nome_produto AS produto,
  pr.preco as preco,
  t.nome_tipo AS tipo_produto,
  ip.quantidade as quantidade,
  (pr.preco * ip.quantidade) AS total_venda

FROM itens_pedido ip

JOIN pedidos p ON ip.id_pedido = p.id_pedido
JOIN clientes c ON p.id_cliente = c.id_cliente
JOIN produtos pr ON ip.id_produto = pr.id_produto
JOIN tipos_produto t ON pr.id_tipo = t.id_tipo;

"""
,con=engine)

df.head()


,cliente,data_pedido,produto,preco,tipo_produto,quantidade,total_venda
0,Enzo Gabriel Monteiro,2025-03-18,Excepturi Reprehenderit,367.57,Ferramentas,4,1470.28
1,Dr. Juan Melo,2024-08-18,Animi Tempore,130.85,Ferramentas,1,130.85
2,João Felipe da Luz,2024-11-08,Expedita Iste,497.62,Higiene,2,995.24
3,João Lucas Fernandes,2025-05-29,Vero Eius,292.34,Livros,5,1461.70
4,Enzo Gabriel Freitas,2025-01-29,Quos Nam,409.85,Ferramentas,2,819.70


In [12]:
# Gerar grafico
fig = px.bar(df, x="cliente", y="total_venda", color="produto", 
             title="Total de Vendas por Cliente e Produto")
fig.show()


In [13]:
deletar_tabelas = 'n'  # Altere para s se quiser dropar

if deletar_tabelas.lower() == 's':
    drop_script = """
    DROP TABLE IF EXISTS itens_pedido CASCADE;
    DROP TABLE IF EXISTS pedidos CASCADE;
    DROP TABLE IF EXISTS produtos CASCADE;
    DROP TABLE IF EXISTS clientes CASCADE;
    DROP TABLE IF EXISTS tipos_produto CASCADE;
    """
    conn = engine.raw_connection()

    try:
        cursor = conn.cursor()
        cursor.execute(drop_script)
        conn.commit()
        print("🗑️ Todas as tabelas foram deletadas com sucesso")
    finally:
        cursor.close()
        conn.close()
else:
    print(f"⚠️ Drop de tabelas não executado. Variável de controle está {deletar_tabelas}")

⚠️ Drop de tabelas não executado. Variável de controle está n


#### **Aula 3 - Stacks Modernas, Data Warehouse, Data Lake e Lakehouse, Data Mesh e Data Fabric**

##### **Modelo Dimensional em DBML – Diagrama DW (dimensões e fato)**

Criar o Diagrama Entidade-Relacionamento (DER) usando o site https://dbdiagram.io/home e o seguinte código DBML (Database Markup Language ou Linguagem de Marcação para Banco de Dados):   

```dbml
Table dim_cliente {
  id_cliente int [pk]
  nome varchar
  idade int
  cidade varchar
}

Table dim_produto {
  id_produto int [pk]
  nome_produto varchar
  categoria varchar
  preco decimal
}

Table fato_pedidos {
  id_pedido int [pk]
  id_cliente int [ref: > dim_cliente.id_cliente]
  id_produto int [ref: > dim_produto.id_produto]
  id_data int [ref: > dim_data.id_data]
  id_regiao int [ref: > dim_regiao.id_regiao]
  data_pedido date
  quantidade int
  valor_total decimal
}

Table dim_data {
  id_data int [pk]
  data date
  ano int
  mes int
  dia int
  dia_semana varchar
  nome_mes varchar
}

Table dim_regiao {
  id_regiao int [pk]
  nome_regiao varchar
  estado varchar
  cidade varchar
}


![Diagrama DW](imagame.png)


##### **Códigos - Carregar dados no PostgreSQL**

In [14]:
# Criar tabelas
ddl_dim_fato = [
    """
    CREATE TABLE IF NOT EXISTS dim_cliente (
      id_cliente INT PRIMARY KEY,
      nome VARCHAR(100),
      idade INT,
      cidade VARCHAR(100)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS dim_produto (
      id_produto INT PRIMARY KEY,
      nome_produto VARCHAR(100),
      categoria VARCHAR(50),
      preco DECIMAL(10,2)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS fato_pedidos (
      id_pedido INT PRIMARY KEY,
      id_cliente INT REFERENCES dim_cliente(id_cliente),
      id_produto INT REFERENCES dim_produto(id_produto),
      data_pedido DATE,
      quantidade INT,
      valor_total DECIMAL(10,2)
    );
    """
]

table_names = [
    "dim_cliente",
    "dim_produto",
    "fato_pedidos"
]

# Validar se as tabelas já existe
with engine.begin() as conn:
    for cmd, table in zip(ddl_dim_fato, table_names):
        result = conn.execute(
            text(f"SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_schema = 'public' AND table_name = '{table}');")
        )
        exists = result.scalar()
        if not exists:
            conn.execute(text(cmd))
            print(f"✅ Tabela '{table}' criada com sucesso!")
        else:
            print(f"⚠️ Tabela '{table}' já existe. Nenhuma alteração feita.")

⚠️ Tabela 'dim_cliente' já existe. Nenhuma alteração feita.
⚠️ Tabela 'dim_produto' já existe. Nenhuma alteração feita.
⚠️ Tabela 'fato_pedidos' já existe. Nenhuma alteração feita.


In [15]:
# Apagar dados das tabelas e executar arquivos .sql
github_urls = [
    "https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/refs/heads/main/Fase%203/Aula4/sql/aula3-dw/insert_dim_cliente.sql",
    "https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/refs/heads/main/Fase%203/Aula4/sql/aula3-dw/insert_dim_produto.sql"

]

delete_script = """
TRUNCATE TABLE dim_cliente, dim_produto CASCADE
"""

with engine.begin() as conn:
    conn.execute(text(delete_script))
    print("🗑️ Dados apagados de todas as tabelas.")

    for url in github_urls:
        response = requests.get(url)
        response.raise_for_status()
        sql_content = response.text
        conn.execute(text(sql_content))
        print(f"✅ Executado: {url.split('/')[-1]}")


🗑️ Dados apagados de todas as tabelas.
✅ Executado: insert_dim_cliente.sql
✅ Executado: insert_dim_produto.sql


In [16]:
# Apagar dados das tabelas e executar arquivos .sql
github_urls = [
    "https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/refs/heads/main/Fase%203/Aula4/sql/aula3-dw/insert_fato_pedidos.sql"
]

delete_script = """
TRUNCATE TABLE fato_pedidos CASCADE
"""

with engine.begin() as conn:
    conn.execute(text(delete_script))
    print("🗑️ Dados apagados de todas as tabelas.")

    for url in github_urls:
        response = requests.get(url)
        response.raise_for_status()
        sql_content = response.text
        conn.execute(text(sql_content))
        print(f"✅ Executado: {url.split('/')[-1]}")


🗑️ Dados apagados de todas as tabelas.
✅ Executado: insert_fato_pedidos.sql


In [17]:
# Join para ver a ultima compra do cliente
df = pd.read_sql_query(

"""
                       
select 
	b.nome, max(a.data_pedido) ultima_compra

from 
	fato_pedidos as a

join dim_cliente as b 
on a.id_cliente = b.id_cliente

group by
	b.nome
	
"""
,con=engine)

df.head()


,nome,ultima_compra
0,Srta. Luna Costa,2024-10-08
1,Vitor Nascimento,2024-12-20
2,Francisco Rezende,2025-04-24
3,Lucas Nascimento,2025-02-05
4,Luiza Duarte,2025-01-02


##### **Códigos - Exportar Postgre para SQL**

In [18]:
# Configurar conexão PostgreSQL
pg_config = {
    "host": host,
    "database": banco,
    "user": usuario,
    "password": senha,
    "port": porta
}

# Configurar dados buckt do AWS S3
bucket_name = "aula-data-lake-430854566059"
s3_prefix = "raw/"

# Listar tabelas a exportar 
tabelas = ['clientes', 'produtos', 'pedidos', 'itens_pedido', 'tipos_produto']

# Validar conexão com PostgreSQL usando o engine
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        versao = result.fetchone()
        print(f"✅ Conexão com o PostgreSQL estabelecida: {versao[0]}\n")
except Exception as e:
    print("❌ Erro ao conectar ao PostgreSQL:", e)
    print()

# Criar conexão com S3
s3 = session.client('s3')
region = s3.meta.region_name or "us-east-1" 

# Verificar buckt
try:
    s3.head_bucket(Bucket=bucket_name)
    print(f"✅ Bucket '{bucket_name}' já existe\n")

except ClientError as e:
    error_code = int(e.response['Error']['Code'])
    if error_code == 404:
        print(f"Bucket '{bucket_name}' não existe, criando...\n")
        if region == "us-east-1":
            s3.create_bucket(Bucket=bucket_name)
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"Bucket '{bucket_name}' criado com sucesso\n")
    else:
        raise

# Exportar dados do Postgres
for tabela in tabelas:
    print(f"Exportando tabela: {tabela}")
    df = pd.read_sql(f"SELECT * FROM {tabela};", engine) 

    # Salvar como CSV em memória
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)

    # Enviar para S3
    s3_key = f"{s3_prefix}{tabela}.csv"
    s3.put_object(Bucket=bucket_name, Key=s3_key, Body=csv_buffer.getvalue())
    print(f"✅ {tabela} salva no S3 em: s3://{bucket_name}/{s3_key}\n")

print("Exportação concluída com sucesso.")


✅ Conexão com o PostgreSQL estabelecida: PostgreSQL 17.4 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 12.4.0, 64-bit

✅ Bucket 'aula-data-lake-430854566059' já existe

Exportando tabela: clientes
✅ clientes salva no S3 em: s3://aula-data-lake-430854566059/raw/clientes.csv

Exportando tabela: produtos
✅ produtos salva no S3 em: s3://aula-data-lake-430854566059/raw/produtos.csv

Exportando tabela: pedidos
✅ pedidos salva no S3 em: s3://aula-data-lake-430854566059/raw/pedidos.csv

Exportando tabela: itens_pedido
✅ itens_pedido salva no S3 em: s3://aula-data-lake-430854566059/raw/itens_pedido.csv

Exportando tabela: tipos_produto
✅ tipos_produto salva no S3 em: s3://aula-data-lake-430854566059/raw/tipos_produto.csv

Exportação concluída com sucesso.


In [19]:
# Criar um dicionário com as opções de armazenamento, especificando o perfil
storage_options = {"profile": "academy"}

# Caminhos no S3
RAW_PREFIX = 's3://aula-data-lake-430854566059/raw/'
SILVER_PREFIX = 's3://aula-data-lake-430854566059/silver/'
GOLD_PREFIX = 's3://aula-data-lake-430854566059/gold/'

# Camada Silver --> Leitura e Limpeza dos Dados
# Uso do storage_options=storage_options para garantir o uso correto das credenciais
clientes = pd.read_csv(RAW_PREFIX + 'clientes.csv', storage_options=storage_options) 
produtos = pd.read_csv(RAW_PREFIX + 'produtos.csv', storage_options=storage_options)
tipos_produto = pd.read_csv(RAW_PREFIX + 'tipos_produto.csv', storage_options=storage_options)
pedidos = pd.read_csv(RAW_PREFIX + 'pedidos.csv', storage_options=storage_options)
itens_pedido = pd.read_csv(RAW_PREFIX + 'itens_pedido.csv', storage_options=storage_options)

# Padronizar colunas para lower case
for df in [clientes, produtos, tipos_produto, pedidos, itens_pedido]:
    df.columns = [col.lower() for col in df.columns]

# Limpeza básica
clientes = clientes.drop_duplicates().dropna(subset=['id_cliente'])
produtos = produtos.drop_duplicates().dropna(subset=['id_produto'])
tipos_produto = tipos_produto.drop_duplicates().dropna(subset=['id_tipo'])
pedidos = pedidos.drop_duplicates().dropna(subset=['id_pedido', 'id_cliente'])
itens_pedido = itens_pedido.drop_duplicates().dropna(subset=['id_item', 'id_pedido', 'id_produto'])

# Padronização de datas
if 'data_pedido' in pedidos.columns:
    pedidos['data_pedido'] = pd.to_datetime(pedidos['data_pedido'], errors='coerce')

# Salvar camada Silver em Parquet no S3
# Uso do storage_options=storage_options para garantir o uso correto das credenciais
clientes.to_parquet(SILVER_PREFIX + 'clientes.parquet', index=False, storage_options=storage_options)
produtos.to_parquet(SILVER_PREFIX + 'produtos.parquet', index=False, storage_options=storage_options)
tipos_produto.to_parquet(SILVER_PREFIX + 'tipos_produto.parquet', index=False, storage_options=storage_options)
pedidos.to_parquet(SILVER_PREFIX + 'pedidos.parquet', index=False, storage_options=storage_options)
itens_pedido.to_parquet(SILVER_PREFIX + 'itens_pedido.parquet', index=False, storage_options=storage_options)

# Camada Gold Layer --> Enriquecimento e Agregações

# Fato de vendas com todas as dimensões
gold_vendas = itens_pedido.merge(pedidos, on='id_pedido') \
    .merge(produtos, on='id_produto') \
    .merge(tipos_produto, on='id_tipo') \
    .merge(clientes, on='id_cliente')

# Valor total do item
if 'quantidade' in gold_vendas.columns and 'preco_unitario' in gold_vendas.columns:
    gold_vendas['valor_total_item'] = gold_vendas['quantidade'] * gold_vendas['preco_unitario']

# Criação do campo anomesdia
if 'data_pedido' in gold_vendas.columns:
    gold_vendas['anomesdia'] = gold_vendas['data_pedido'].dt.strftime('%Y%m%d')

# Resumo por cliente
gold_vendas_por_cliente = gold_vendas.groupby(['id_cliente', 'nome']) \
    .agg({'valor_total_item': 'sum', 'id_pedido': 'nunique'}) \
    .rename(columns={'valor_total_item': 'valor_total_comprado', 'id_pedido': 'num_pedidos'}) \
    .reset_index()

# Resumo por tipo de produto
gold_vendas_por_tipo = gold_vendas.groupby(['id_tipo', 'nome_tipo']) \
    .agg({'valor_total_item': 'sum', 'quantidade': 'sum'}) \
    .rename(columns={'valor_total_item': 'total_vendido', 'quantidade': 'quantidade_total'}) \
    .reset_index()

# Salvar camada Gold no S3 particionado por anomesdia
gold_vendas.to_parquet(GOLD_PREFIX + 'fato_vendas/', partition_cols=['anomesdia'], index=False, storage_options=storage_options)
gold_vendas_por_cliente.to_parquet(GOLD_PREFIX + 'vendas_por_cliente.parquet', index=False, storage_options=storage_options)
gold_vendas_por_tipo.to_parquet(GOLD_PREFIX + 'vendas_por_tipo.parquet', index=False, storage_options=storage_options)

print("Processamento das camadas Silver e Gold no S3 particionado por ano mes e dia (anomesdia) finalizado com sucesso!")


c:\Users\ricar\anaconda3\envs\ambiente_fase3\Lib\site-packages\fsspec\registry.py:298: UserWarning:

Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.




Processamento das camadas Silver e Gold no S3 particionado por ano mes e dia (anomesdia) finalizado com sucesso!


#### **Aula 4 - Linguagem SQL Para Analise, Seleção, Filgram, Agrupamento e Joins**

In [ ]:
# Exemplo de uso --> Sem a utilizar a criação de uma conexão
aws_profile_name = "academy"  

credentials = session.get_credentials()
region = session.region_name or 'us-east-1' 

duckdb.sql("INSTALL httpfs;")
duckdb.sql("LOAD httpfs;")
duckdb.sql(f"SET s3_region='{region}';")
duckdb.sql(f"SET s3_access_key_id='{credentials.access_key}';")
duckdb.sql(f"SET s3_secret_access_key='{credentials.secret_key}';")

# Se estiver usando credenciais temporárias, o token também será necessário
if credentials.token:
    duckdb.sql(f"SET s3_session_token='{credentials.token}';")

s3_path = "s3://aula-data-lake-430854566059/gold/vendas_por_cliente.parquet"

query = f"""
SELECT 
    *
FROM 
    read_parquet('{s3_path}')
    
LIMIT 
    5;
"""

resultado_df = duckdb.sql(query).df()
print(resultado_df)


   id_cliente            nome  valor_total_comprado  num_pedidos
0           1     Yago Novaes               1962.73            1
1           2  Isadora Castro                817.44            1
2           6     Raul da Luz               1230.50            1
3           7    Clara Araújo                843.70            1
4           8   Yuri Teixeira               5013.49            2


In [22]:
# Exemplo de uso --> Utilizar a criação de uma conexão
# --- ETAPA 1: Carregar credenciais da AWS (parte do seu código original) ---
aws_profile_name = "academy" 

# Criamos a sessão para obter as credenciais
session = boto3.Session(profile_name=aws_profile_name)
credentials = session.get_credentials()
region = session.region_name or 'us-east-1' 

# --- ETAPA 2: ADAPTAÇÃO DO CÓDIGO ---

# 2.1. Em vez de comandos "SET", criamos um dicionário de configuração
duckdb_config = {
    's3_region': region,
    's3_access_key_id': credentials.access_key,
    's3_secret_access_key': credentials.secret_key
}
# Adiciona o token de sessão ao dicionário, se ele existir
if credentials.token:
    duckdb_config['s3_session_token'] = credentials.token

# 2.2. Criamos um objeto de conexão explícito, passando o dicionário
#     O banco de dados ':memory:' significa que ele rodará em memória RAM.
con = duckdb.connect(database=':memory:', config=duckdb_config)

# 2.3. Usamos o objeto 'con' para instalar/carregar a extensão
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")

# A definição do caminho e da query continua a mesma
s3_path = "s3://aula-data-lake-430854566059/gold/vendas_por_cliente.parquet"

query = f"""
SELECT 
    *
FROM 
    read_parquet('{s3_path}')
    
LIMIT 
    5;
"""

# 2.4. Executamos a consulta usando a conexão 'con'
resultado_df = con.execute(query).df()

# 2.5. Fechamos a conexão após o uso
con.close()

# Exibe o resultado final
print(resultado_df)

   id_cliente            nome  valor_total_comprado  num_pedidos
0           1     Yago Novaes               1962.73            1
1           2  Isadora Castro                817.44            1
2           6     Raul da Luz               1230.50            1
3           7    Clara Araújo                843.70            1
4           8   Yuri Teixeira               5013.49            2


In [ ]:
# Teste Professor

session = boto3.Session(profile_name="academy")
credentials = session.get_credentials().get_frozen_credentials()
region = session.region_name or "us-east-1"

DIM_PREFIX = "s3://aula-data-lake-430854566059/gold/"

duckdb_config = {
    "s3_region": region,
    "s3_access_key_id": credentials.access_key,
    "s3_secret_access_key": credentials.secret_key,
}
if credentials.token:
    duckdb_config["s3_session_token"] = credentials.token

con = duckdb.connect(config=duckdb_config)

df = con.execute(f"""
    SELECT *
    FROM read_parquet('{DIM_PREFIX}vendas_por_cliente')
""").fetchdf()

print(df.head())

   id_cliente            nome  valor_total_comprado  num_pedidos
0           1     Yago Novaes               1962.73            1
1           2  Isadora Castro                817.44            1
2           6     Raul da Luz               1230.50            1
3           7    Clara Araújo                843.70            1
4           8   Yuri Teixeira               5013.49            2


In [ ]:
df = con.execute(f"""
    SELECT *
    FROM read_parquet('{DIM_PREFIX}fato_vendas/anomesdia=20240620/*.parquet')
""").fetchdf()





print(df.head())

   id_item  id_pedido  id_produto  quantidade  preco_unitario data_pedido  \
0       35         26          19           1           32.64  2024-06-20   
1       72         26           3           5           84.67  2024-06-20   
2       35         26          19           1           32.64  2024-06-20   
3       72         26           3           5           84.67  2024-06-20   
4       35         26          19           1           32.64  2024-06-20   

      status  id_cliente             nome_produto   preco  id_tipo  \
0  Concluído           8  Excepturi Reprehenderit  367.57        9   
1  Concluído           8    Cupiditate Reiciendis  119.37        2   
2  Concluído           8  Excepturi Reprehenderit  367.57        9   
3  Concluído           8    Cupiditate Reiciendis  119.37        2   
4  Concluído           8  Excepturi Reprehenderit  367.57        9   

     nome_tipo           nome              email         telefone cidade  \
0  Ferramentas  Yuri Teixeira  uduarte@i

In [ ]:
session = boto3.Session(profile_name="academy")
credentials = session.get_credentials().get_frozen_credentials()
region = session.region_name or "us-east-1"

DIM_PREFIX = "s3://aula-data-lake-430854566059/gold/"

duckdb_config = {
    "s3_region": region,
    "s3_access_key_id": credentials.access_key,
    "s3_secret_access_key": credentials.secret_key,
}
if credentials.token:
    duckdb_config["s3_session_token"] = credentials.token

con = duckdb.connect(config=duckdb_config)



con.sql("""
    CREATE TEMP TABLE top_clientes AS
    SELECT *
    FROM read_parquet('s3://aula-data-lake-430854566059/gold/vendas_por_cliente')

    
""")

# Consulta usando a tabela temporária
df = con.sql("SELECT * FROM top_clientes").df()

df.head()



,id_cliente,nome,valor_total_comprado,num_pedidos
0,1,Yago Novaes,1962.73,1
1,2,Isadora Castro,817.44,1
2,6,Raul da Luz,1230.50,1
3,7,Clara Araújo,843.70,1
4,8,Yuri Teixeira,5013.49,2


#### **Aula 5 - Tipos de Bancos NoSQL: Documentos, Chave-Valor, Grafos e Colunas**